In [3]:
# %matplotlib inline
# %matplotlib notebook
import tqdm, copy
import random, os
import pandas as pd
from tqdm import trange
import matplotlib.pyplot as plt
from collections import defaultdict
from overcooked_ai_py.utils import save_pickle
from human_aware_rl.utils import set_global_seed
from human_aware_rl.human.process_dataframes import *
# from human_aware_rl.static import *
import pickle as pkl

set_global_seed(1884)
import ast
import json

from scipy import stats

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
import os
from ast import literal_eval
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas


# Define Helper Functions to Open Data

In [4]:
def import_cleaned_data(filename):
    

    with open(filename,'rb') as file:
        data = pkl.load(file)

    return data




# Clean Train Trials

In [27]:
filename = '../data/human/anonymized/clean_test_trials.pkl'
clean_data = import_cleaned_data(filename)


In [28]:
clean_data.head()

,Unnamed: 0,Unnamed: 0.1,cur_gameloop,datetime,is_leader,joint_action,layout,layout_name,next_state,reward,...,state,time_elapsed,time_left,is_wait,completed,run,workerid_num,reward_norm,cur_gameloop_total,reward_norm_total
7215,7215,22170,0.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.151,180.0,True,True,run-1,1,0.0,1204,640.0
7216,7216,22171,1.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.303,180.0,True,True,run-1,1,0.0,1204,640.0
7217,7217,22172,2.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.451,180.0,True,True,run-1,1,0.0,1204,640.0
7218,7218,22173,3.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.601,179.0,True,True,run-1,1,0.0,1204,640.0
7219,7219,22174,4.0,1.557859e+12,True,"[[0, 0], [0, 0]]","['XXXXXXXXX', 'O XSXOX S', 'X P 1 X', 'X2 P...",asymmetric_advantages,"{'players': [{'position': [6, 2], 'orientation...",0.0,...,"{'players': [{'position': [6, 2], 'orientation...",0.752,179.0,True,True,run-1,1,0.0,1204,640.0


In [29]:
clean_data.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'cur_gameloop', 'datetime', 'is_leader',
       'joint_action', 'layout', 'layout_name', 'next_state', 'reward',
       'round_num', 'round_type', 'score', 'state', 'time_elapsed',
       'time_left', 'is_wait', 'completed', 'run', 'workerid_num',
       'reward_norm', 'cur_gameloop_total', 'reward_norm_total'],
      dtype='object')

In [30]:
clean_data['layout_name'].unique()

array(['asymmetric_advantages', 'coordination_ring', 'cramped_room',
       'random0', 'random3'], dtype=object)

In [31]:
fc_data = clean_data[clean_data['layout_name']=='random0']

In [32]:
fc_data['run'].unique()

array(['run-1', 'run-2'], dtype=object)

In [33]:
fc_data['workerid_num'].unique()

array([ 1,  3, 10, 11, 12, 13])

In [34]:
fc_data.groupby(['run','workerid_num', 'reward_norm_total']).size().reset_index().rename(columns={0:'count'})



,run,workerid_num,reward_norm_total,count
0,run-1,1,380.0,1204
1,run-1,3,360.0,1204
2,run-2,10,440.0,1204
3,run-2,11,400.0,1204
4,run-2,12,160.0,1204
5,run-2,13,300.0,1204


# Clean Human AI Trials

In [11]:
filename = '../data/human/anonymized/clean_human_AI_trials.pkl'
human_ai_trials_data = import_cleaned_data(filename)


In [12]:
human_ai_trials_data

,action,cur_gameloop,datetime,indexOf,joint_action,layout,layout_name,model_type,next_state,player_index,...,score,state,template,time_elapsed,time_left,viewTime,reward_norm,cur_gameloop_total,reward_norm_total,workerid_num
482,NaN,0.0,1.558577e+12,NaN,"[[0, 0], [1, 0]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [1, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [1, 2], 'orientation...",NaN,2.062,58.0,NaN,0.0,393,0.0,11
483,NaN,1.0,1.558577e+12,NaN,"[[1, 0], INTERACT]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [1, 2], 'orientation...",NaN,2.089,58.0,NaN,0.0,393,0.0,11
484,NaN,2.0,1.558577e+12,NaN,"[[0, 0], [-1, 0]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.121,58.0,NaN,0.0,393,0.0,11
485,NaN,3.0,1.558577e+12,NaN,"[[0, 0], [0, -1]]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.273,58.0,NaN,0.0,393,0.0,11
486,NaN,4.0,1.558577e+12,NaN,"[[0, 0], INTERACT]","[XXPXX, O 2O, X1 X, XDXSX]",cramped_room,pbt,"{'players': [{'position': [2, 2], 'orientation...",0,...,0.0,"{'players': [{'position': [2, 2], 'orientation...",NaN,2.416,58.0,NaN,0.0,393,0.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137365,NaN,398.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.048,0.0,NaN,0.0,403,0.0,55
137366,NaN,399.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.199,0.0,NaN,0.0,403,0.0,55
137367,NaN,400.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.341,0.0,NaN,0.0,403,0.0,55
137368,NaN,401.0,1.558584e+12,NaN,"[[0, 0], INTERACT]","[XXXPX, O X1P, O2X X, D X X, XXXSX]",random0,ppo_sp,"{'players': [{'position': [3, 1], 'orientation...",0,...,0.0,"{'players': [{'position': [3, 1], 'orientation...",NaN,60.489,0.0,NaN,0.0,403,0.0,55
